<a href="https://colab.research.google.com/github/alirezash97/BraTS/blob/master/Survival.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [87]:
import tensorflow as tf
import os
import numpy as np
from nibabel.testing import data_path
import nibabel as nib
import matplotlib.pyplot as plt
from keras.utils import to_categorical
import cv2
import keras 
import random

In [19]:
import glob, os
images_path = glob.glob('/content/drive/My Drive/BRATS2020/MICCAI_BraTS2020_TrainingData/**/*.nii.gz', recursive=True)

In [88]:

X_trainset_filenames = []

for item in images_path:
  if 'seg' in item:
    X_trainset_filenames.append(os.path.join(data_path, item))
  else:
    pass

In [89]:
print(len(X_trainset_filenames))

369


In [91]:
def shuffle(input_path):

  a_list = list(range(0, len(input_path)))
  random.shuffle(a_list)


  new_input_path = []
  for i in a_list:
    
    new_input_path.append(input_path[i])

  return new_input_path

In [101]:
 X_trainset_filenames = shuffle( X_trainset_filenames)

In [100]:

def load_case(label_nifty_file):
    
    # load the label file, get the image content and return a numpy array for each

    label = np.array(nib.load(label_nifty_file).get_fdata())

    for i in range(240):
          for j in range(240):
            for k in range(155):
              if label[i, j, k] == 4.0 :
                label[i, j, k] = 3.0
              else:
                pass
    
    
    return label

In [102]:


from ipywidgets import interact

M = load_case(X_trainset_filenames[0])

def explore_3dimage(layer):
    plt.figure(figsize=(10, 5))
    channel = 3
    plt.imshow(M[:, :, layer]);
    plt.title('Explore Layers of Tumor prediction', fontsize=20)
    plt.axis('off')
    return layer

# Run the ipywidgets interact() function to explore the data
interact(explore_3dimage, layer=(0, 155 - 1));

interactive(children=(IntSlider(value=77, description='layer', max=154), Output()), _dom_classes=('widget-inte…

In [112]:

## get the most important part for survival task (the tumor)


def get_max(image, out_x, out_y, out_z, stride, number_of_images_to_extract):
  
  x = image.shape[0]
  y = image.shape[1]
  z = image.shape[2]


  total_number_of_cubes = ( int((z - out_z)/stride) * int((x - out_x)/stride) * int((y - out_y)/stride) )
  details = np.zeros((total_number_of_cubes, 4))

  counter = 0
  for i in range( int((z - out_z)/stride) ):
    for j in range( int((x - out_x)/stride) ):
      for k in range( int((y - out_y)/stride) ):

        
        temp = np.sum(image[ (j*stride):((j*stride)+out_x), (k*stride):((k*stride)+out_y), (i*stride):((i*stride)+out_z) ])
        details[counter] = (j*stride), (k*stride), (i*stride), temp
        counter += 1

  

  
  final_extracted_images = []
  for l in range(number_of_images_to_extract):
    
    max_ind = np.unravel_index(np.argmax(details, axis=None), details.shape) 
  
    x_start = int(details[max_ind[0], 0])
    y_start = int(details[max_ind[0], 1])
    z_start = int(details[max_ind[0], 2])

    temp = image[ x_start:(x_start+out_x) , y_start:(y_start+out_y) , z_start:(z_start+out_z) ]
    final_extracted_images.append(temp)
    details[max_ind[0], 3] = 0



  return np.array(final_extracted_images)




In [113]:
image = get_max(M, 64, 64, 64, 8, 5)

In [117]:
print(image.shape)

(5, 64, 64, 64)


In [116]:
from ipywidgets import interact



def explore_3dimage(layer):
    plt.figure(figsize=(10, 5))
    channel = 3
    plt.imshow(image[2, :, :, layer]);
    plt.title('Explore Layers of Tumor prediction', fontsize=20)
    plt.axis('off')
    return layer

# Run the ipywidgets interact() function to explore the data
interact(explore_3dimage, layer=(0, 64 - 1));

interactive(children=(IntSlider(value=31, description='layer', max=63), Output()), _dom_classes=('widget-inter…

In [105]:
import pandas as pd 
pd.read_csv("/content/drive/My Drive/BRATS2020/MICCAI_BraTS2020_TrainingData/survival_info.csv")

,Brats20ID,Age,Survival_days,Extent_of_Resection
0,BraTS20_Training_001,60.463,289,GTR
1,BraTS20_Training_002,52.263,616,GTR
2,BraTS20_Training_003,54.301,464,GTR
3,BraTS20_Training_004,39.068,788,GTR
4,BraTS20_Training_005,68.493,465,GTR
...,...,...,...,...
231,BraTS20_Training_363,57.000,62,GTR
232,BraTS20_Training_366,72.000,633,GTR
233,BraTS20_Training_367,60.000,437,STR
234,BraTS20_Training_368,49.000,442,GTR
